In [2]:
from autogen import AssistantAgent, UserProxyAgent, GroupChatManager, GroupChat
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent, TEXT_FORMATS

In [13]:
#TODO: future config_file should be a json file to not repeat this lines
# This list will 'connect' to the llama3.1 model running on llm container
config_list = [{
        "model": "llama3.1", # The name of the used model (to run the code below it must support the using of tools!)
        "base_url": "http://localhost:11434/v1",
        "api_key": "ollama" # Asked but not used
}]

llm_config = {"config_list": config_list, "temperature": 0.2}

prompt = """You are a data scientist that works with Cypher queries.
All you have to do is translate the given answer as cypher queries.
You mustn't use your previous knowledge,you must only use the pdf manual i will pass to you.

You must generate the easiest query possible.
You must be precise and give all the information you have. If you think you are not correct you must re-generate the query.
When you think the query can be run you can submit it as the final answer.
When you have written your final answer you must send 'TERMINATE'.
"""

print(f'Accepted formats for "docs_path": \n{TEXT_FORMATS}')

Accepted formats for "docs_path": 
['txt', 'json', 'csv', 'tsv', 'md', 'html', 'htm', 'rtf', 'rst', 'jsonl', 'log', 'xml', 'yaml', 'yml', 'pdf']


In [21]:

def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()


ragagent = RetrieveUserProxyAgent(
    name="ragagent",
    is_termination_msg=termination_msg,
    max_consecutive_auto_reply=3,
    human_input_mode="NEVER",
    system_message="You are an assistant that will help the coder to generate the right cypher query.",
    retrieve_config={
        "task": "code",
        "docs_path": ["https://s3.amazonaws.com/artifacts.opencypher.org/openCypher9.pdf", "https://piazza.com/class_profile/get_resource/jsd7vhgcid7jj/jvyxpxy1wss6y3"],
        "chunk_token_size": 1000,
        "model": config_list[0]["model"],
        "get_or_create": True,
    },
    code_execution_config=False,  # we don't want to execute code in this case.
    description="Assistant who has extra content retrieval power for solving difficult problems.",
)

coder = AssistantAgent(
    name="coder",
    is_termination_msg=termination_msg,
    system_message=prompt,
    llm_config=llm_config,
    human_input_mode="NEVER"
)

PROBLEM = "\n\nQUESTION:\nHow can i find the smallest path from one node to another one?"


ragagent.reset()
coder.reset()

ragagent.initiate_chat(coder, message=prompt+PROBLEM, max_turns=5)

ragagent (to coder):

You are a data scientist that works with Cypher queries.
All you have to do is translate the given answer as cypher queries.
You mustn't use your previous knowledge,you must only use the pdf manual i will pass to you.

You must generate the easiest query possible.
You must be precise and give all the information you have. If you think you are not correct you must re-generate the query.
When you think the query can be run you can submit it as the final answer.
When you have written your final answer you must send 'TERMINATE'.


QUESTION:
How can i find the smallest path from one node to another one?

--------------------------------------------------------------------------------
[autogen.oai.client: 09-03 13:53:46] {329} WARNING - Model llama3.1 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
coder (to ragagent):

I've reviewed the Cypher manual and I'm ready 

ChatResult(chat_id=None, chat_history=[{'content': "You are a data scientist that works with Cypher queries.\nAll you have to do is translate the given answer as cypher queries.\nYou mustn't use your previous knowledge,you must only use the pdf manual i will pass to you.\n\nYou must generate the easiest query possible.\nYou must be precise and give all the information you have. If you think you are not correct you must re-generate the query.\nWhen you think the query can be run you can submit it as the final answer.\nWhen you have written your final answer you must send 'TERMINATE'.\n\n\nQUESTION:\nHow can i find the smallest path from one node to another one?", 'role': 'assistant', 'name': 'ragagent'}, {'content': "I've reviewed the Cypher manual and I'm ready to generate a query. \n\nTo find the smallest path between two nodes, we need to use the shortestPath function in Cypher.\n\nHere is my attempt at generating the query:\n\n```cypher\nMATCH p = shortestPath((a)-[*..10]-(b))\nWHER